In [ ]:
# Task: I received a list of about 1500 US investment banks. My tasks are:
# 1- identify all IP4s used by those investment banks.
# 2- get all EDGAR Log records from the US SEC website to verify those banks' digital footprints.
# 3- filter edgar log records associated with those IP4s.

In [ ]:
import pandas as pd
import os
import json
import urllib.request
import requests
from io import BytesIO
from urllib.request import urlopen
from bs4 import BeautifulSoup
import glob

In [ ]:
download_dir= "./ip4_json_dl/"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)


def json_dowloader(org):
    path = # add path here
    root = 'https://ip-netblocks.whoisxmlapi.com/api/v2?apiKey=API KEY HERE'
    url=root + org
    file=requests.get(url)
    data=file.json()
    with open(download_dir+org+'.json','w') as f:
        json.dump(data,f)

# download json files
if __name__ == '__main__':
    path = #add path
    df = pd.read_excel(os.path.join(path, 'filename.xlsx'))

    org_list = df['org'].to_list()
    print(len(org_list))
    print(org_list[:20])

    for org in org_list:
        try:
            f = json_dowloader(org)
        except:
            log = os.path.join(download_dir, "ip_dl_log.txt")
            f_log = open(log, "a")
            f_log.write('download_error:' + org + "\n")
            f_log.close()

In [ ]:
# get IP4 function
def get_ip(file):
    try:
        with open(file, 'r') as f:
            data = json.load(f)
            search = data['search']
            print(search)
            count = data['result']['count']
            print(count)
            if count:
                orgs = data['result']['inetnums']
                # print(orgs)
                ip_ranges = []
                modified_dates = []
                names = []
                emails = []
                phones = []
                countries = []
                cities = []
                postcodes = []
                addresses = []
                sources = []
                for i in range(0, len(orgs)):
                    ip_range = orgs[i]['inetnum']
                    ip_ranges.append(ip_range)
                    modified_date = orgs[i]['modified']
                    modified_dates.append(modified_date)
                    source = orgs[i]['source']
                    sources.append(source)
                    if orgs[i]['org'] is not None:
                        name = orgs[i]['org']['name']
                        email = orgs[i]['org']['email']
                        phone = orgs[i]['org']['phone']
                        country = orgs[i]['org']['country']
                        city = orgs[i]['org']['city']
                        postal_code = orgs[i]['org']['postalCode']
                        address = orgs[i]['org']['address']
                    if orgs[i]['org'] is None:
                        name = "NA"
                        email = "NA"
                        phone = "NA"
                        country = "NA"
                        city = "NA"
                        postal_code = "NA"
                        address = "NA"

                    names.append(name)
                    emails.append(email)
                    phones.append(phone)
                    countries.append(country)
                    cities.append(city)
                    postcodes.append(postal_code)
                    addresses.append(address)
            df = pd.DataFrame({'registered_name': names,
                                   'ip_range': ip_ranges,
                                   'source': sources,
                                   'modified_date': modified_dates,
                                   'email': emails,
                                   'phone': phones,
                                   'address': addresses,
                                   'postal_code': postcodes,
                                   'city': cities,
                                   'country': countries,
                                   })
            df['searched_name'] = search
            df['found_count'] = count
            return df
    except:
        log = os.path.join(download_dir, "0_json_read_log.txt")
        f_log = open(log, "a")
        f_log.write('read_error:' + filename + "\n")
        f_log.close()


# get IP4        
output_dir = # add path here
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
path = # add path here
all_files = glob.glob(os.path.join(path, "*.json"))
for file in all_files:
    basename = os.path.basename(file)
    filename = basename.replace(".json", "").strip()
    df = get_ip(file)
    if df is not None:
        df.to_csv(output_dir + filename + ".csv", index=False)
    else:
        log = os.path.join(output_dir, "0_json_extract_log.txt")
        f_log = open(log, "a")
        f_log.write('no result:' + filename + "\n")
        f_log.close()

In [ ]:
# merge all the csv files
def merge_csv(path):
    all_files = glob.glob(os.path.join(path, "*.csv"))
    df_each_file = (pd.read_csv(f) for f in all_files)
    concatenated_df = pd.concat(df_each_file, ignore_index=True)
    concatenated_df.to_csv(os.path.join(path, 'filename.csv'), index=False)

# execute
path = # add path here
merge_csv(path)

In [ ]:
#phase 4 - filtering
if __name__=='__main__':
    path = # add path here
    df = pd.read_csv(os.path.join(path, 'filename.csv'))
    print(df.columns)
    print("df:", df.shape) 

    df2 = df[df['country'] == 'US']
    print("df2:", df2.shape)  

    df3 = df2[df2['registered_name'] != " "]
    print("df3:", df3.shape) 

    df3['address'] = df3['address'].str.replace("'", "")
    df3['phone'] = df3['phone'].str.replace("-", " ").replace("=", "+")
    df3[['ip_start', 'ip_end']] = df3.ip_range.str.split("-", expand=True)
    df3['org_id'] = df3.groupby('searched_name').ngroup()  

    print(df3.columns)
    print(df3)

    df3.to_csv(os.path.join(path, 'filename.csv'), index=False)

In [ ]:
#phase5 - create IP3 range of each bank because EDGAR log only reviews partial IP4s 123.456.789.xxx
if __name__=='__main__':
    path = # add path here
    df = pd.read_csv(os.path.join(path, 'filename.csv'))
    df2 = df[['org_id', 'searched_name', 'registered_name', 'ip_start', 'ip_end']]

    ip3_start_col = []
    ip3_end_col = []
    for index, row in df2.iterrows():
        if (len(row['ip_start']) <= 16) & len(row['ip_end']) <= 16:
            print(index)
            ip3_start_l = row['ip_start'].split('.')
            ip3_end_l = row['ip_end'].split('.')
            ip3_start = ".".join(ip3_start_l[:3]).strip(" ")
            ip3_end = ".".join(ip3_end_l[:3]).strip(" ")
        else:
            ip3_start = " "
            ip3_end = " "
        ip3_start_col.append(ip3_start)
        ip3_end_col.append(ip3_end)
    df2['ip3_start'] = ip3_start_col
    df2['ip3_end'] = ip3_end_col
    df2 = df2[df2['ip3_start'] != " "]
    # df2['ip3_start']=df2['ip3_start'].astype(float)
    # df2['ip3_end']=df2['ip3_end'].astype(float)
    df2.to_excel(os.path.join(path, 'filename.xlsx'), index=False)

In [ ]:
# get all ip3 in a range
def ip_to_int(ip):
    val = 0
    for i, s in enumerate(ip.split('.')):
        val += int(s) * 256 ** (2 - i)
    return val

def int_to_ip(val):
    octets = []
    for i in range(3):
        octets.append(str(val % 256))
        val = val >> 8
    return '.'.join(reversed(octets))

def findIPs(start, end):
    for i in range(ip_to_int(start), ip_to_int(end) + 1):
        yield int_to_ip(i)

if __name__== '__main__':
    path = # add path here
    df = pd.read_excel(os.path.join(path, 'filename.xlsx'))

    ip3_list = []
    for index, row in df.iterrows():
        print(row["ip3_start"], row["ip3_end"])
        try:
            ip3 = list(findIPs(row["ip3_start"], row["ip3_end"]))
        except:
            ip4 = " "
        print(ip3)
        ip3_list.append(ip3)
    df['ip4'] = ip3_list

    df2=df[df['ip4']!=" "]
    df2.to_excel(os.path.join(path, 'filename.xlsx'), index=False)

#explode the ip3 list
if __name__== '__main__':
    path = # add path here
    df = pd.read_excel(os.path.join(path, 'filename.xlsx'))
    df2 = df.assign(ip4=df.ip4.str.split(",")).explode('ip4')
    df2['ip4']=df2['ip4'].apply(lambda x: x.replace("[","").replace("]","").replace("'",""))
    df2.to_excel(os.path.join(path, 'filename.xlsx'), index=False)

In [ ]:
#get EDGAR Log records associated with the truncated IP4s from 2 Terabytes data from EDGAR Log.
def get_ip3_unique_list(df):
    ip3_list = df['ip3_unpacked'].to_list()
    # print(len(ip3_list)) #203485 ip3
    ip3_set = set(ip3_list)
    ip3_list_unique = list(ip3_set)
    return ip3_list_unique

def get_ip3_truncated(df):
    # get ip3 truncated
    ip_truncated = []
    for row in df['ip']:
        size = len(row)
        mod_ip = row[:size - 4]
        # print(mod_ip)
        ip_truncated.append(mod_ip)
        # look for ip3 of search parquet file
    df['ip'] = ip_truncated
    return df

if __name__== '__main__':
    path = # add path here
    df = pd.read_excel(os.path.join(path, 'filename.xlsx'))
    ip3_l = get_ip3_unique_list(df)

if __name__== '__main__':
    download_dir = "./add path/"
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

# I have extracted about 2 Terabytes data from EDGAR Log. 
if __name__== '__main__':
    path1 = # add path here
    all_files = glob.glob(os.path.join(path1, "*.parq"))
    for file in all_files:
        print(file)
        basename = os.path.basename(file)
        filename = basename.replace(".parq", "").strip()
        try:
            print("Extracted_parquet:", filename)
            df = pd.read_parquet(file)
            df2 = get_ip3_truncated(df)
            df_filtered = df2[df2['ip'].isin(ip3_l)]
            df_filtered.to_csv(download_dir + filename + ".csv", index=False)

        except:
            log = os.path.join(download_dir, "filename.txt")
            f_log = open(log, "a")
            f_log.write('download_error:' + filename + "\n")
            f_log.close()
            print("Error:", filename)